In [1]:
import warnings
warnings.filterwarnings('ignore')
from talib import (RSI, BBANDS, MACD, NATR, WILLR, WMA, EMA, SMA, CCI, CMO, MACD, PPO, ROC, ADOSC, ADX, MOM)
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.regression.rolling import RollingOLS
import statsmodels.api as sm
import pandas_datareader.data as web
import pandas as pd
import numpy as np
from pathlib import Path

%matplotlib inline

DATA_STORE = 'data/assets.h5'
MONTH = 21
YEAR = 12 * MONTH
START = '2000-01-01'
END = '2022-12-31'

sns.set_style('whitegrid')
idx = pd.IndexSlice
T = [1, 5, 10, 21, 42, 63]

results_path = Path('results', 'cnn_for_trading')
if not results_path.exists():
    results_path.mkdir(parents=True)

In [7]:
# Loading Quandl Wiki Stock Prices & Meta Data
adj_ohlcv = ['adj_open', 'adj_close', 'adj_low', 'adj_high', 'adj_volume']
with pd.HDFStore(DATA_STORE) as store:
    prices = (store['quandl/wiki/prices'].loc[idx[START:END, :], adj_ohlcv]
              .rename(columns=lambda x: x.replace('adj_', '')).swaplevel().sort_index().dropna())
    metadata = (store['us_equities/stocks'].loc[:, ['marketcap', 'sector']])
ohlcv = prices.columns.tolist()
prices.volume /= 1e3
prices.index.names = ['symbol', 'date']
metadata.index.name = 'symbol'

In [8]:
ohlcv

['open', 'close', 'low', 'high', 'volume']

In [10]:
# Rolling universe: pick 500 most-traded stocks
dollar_vol = prices.close.mul(prices.volume).unstack('symbol').sort_index()
years = sorted(np.unique([d.year for d in prices.index.get_level_values('date').unique()]))
train_window = 5 # years
universe_size = 500

universe = []
for i, year in enumerate(years[5:], 5):
    start = str(years[i-5])
    end = str(years[i])
    most_traded = (dollar_vol.loc[start:end, :].dropna(thresh=1000, axis=1).median().nlargest(universe_size)
                   .index)
    universe.append(prices.loc[idx[most_traded, start:end], :])

universe = pd.concat(universe)
universe = universe.loc[~universe.index.duplicated()]
universe.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2576938 entries, ('BRK_A', Timestamp('2000-01-03 00:00:00')) to ('BWLD', Timestamp('2018-02-05 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   open    2576938 non-null  float64
 1   close   2576938 non-null  float64
 2   low     2576938 non-null  float64
 3   high    2576938 non-null  float64
 4   volume  2576938 non-null  float64
dtypes: float64(5)
memory usage: 108.3+ MB


In [11]:
universe.groupby('symbol').size().describe()

count     744.000000
mean     3463.626344
std      1173.658096
min      1007.000000
25%      2323.000000
50%      3834.000000
75%      4586.000000
max      4587.000000
dtype: float64

In [12]:
universe.to_hdf('data/universe_data.h5', 'universe')

In [13]:
# Generate Technical Indicators Factors
T = list(range(6, 21))

# Relative Strength Index
for t in T:
    universe[f'{t:02}_RSI'] = universe.groupby(level='symbol').close.apply(RSI, timeperiod=t)

# Williams %R
for t in T:
    universe[f'{t:02}_WILLR'] = (universe.groupby(level='symbol', group_keys=False)
     .apply(lambda x: WILLR(x.high, x.low, x.close, timeperiod=t)))

# Compute Bollinger Bands
def compute_bb(close, timeperiod):
    high, mid, low = BBANDS(close, timeperiod=timeperiod)
    return pd.DataFrame({f'{timeperiod:02}_BBH': high, f'{timeperiod:02}_BBL': low}, index=close.index)
for t in T:
    bbh, bbl = f'{t:02}_BBH', f'{t:02}_BBL'
    universe = (universe.join(universe.groupby(level='symbol').close.apply(compute_bb, timeperiod=t)))
    universe[bbh] = universe[bbh].sub(universe.close).div(universe[bbh]).apply(np.log1p)
    universe[bbl] = universe.close.sub(universe[bbl]).div(universe.close).apply(np.log1p)

In [14]:
# Normalized Average True Range
for t in T:
    universe[f'{t:02}_NATR'] = universe.groupby(level='symbol', group_keys=False).apply(lambda x:
                                                        NATR(x.high, x.low, x.close, timeperiod=t))

# Percentage Price Oscillator
for t in T:
    universe[f'{t:02}_PPO'] = universe.groupby(level='symbol').close.apply(PPO, fastperiod=t, matype=1)

# Moving Average Convergence/Divergence
def compute_macd(close, signalperiod):
    macd = MACD(close, signalperiod=signalperiod)[0]
    return (macd - np.mean(macd))/np.std(macd)

for t in T:
    universe[f'{t:02}_MACD'] = (universe.groupby('symbol', group_keys=False).close
                  .apply(compute_macd, signalperiod=t))

# Momentum
for t in T:
    universe[f'{t:02}_MOM'] = universe.groupby(level='symbol').close.apply(MOM, timeperiod=t)

In [15]:
# Weighted Moving Average
for t in T:
    universe[f'{t:02}_WMA'] = universe.groupby(level='symbol').close.apply(WMA, timeperiod=t)

# Exponential Moving Average
for t in T:
    universe[f'{t:02}_EMA'] = universe.groupby(level='symbol').close.apply(EMA, timeperiod=t)

# Commodity Channel Index
for t in T:
    universe[f'{t:02}_CCI'] = (universe.groupby(level='symbol', group_keys=False)
     .apply(lambda x: CCI(x.high, x.low, x.close, timeperiod=t)))

# Chande Momentum Oscillator
for t in T:
    universe[f'{t:02}_CMO'] = universe.groupby(level='symbol').close.apply(CMO, timeperiod=t)

In [16]:
# Rate of Change
for t in T:
    universe[f'{t:02}_ROC'] = universe.groupby(level='symbol').close.apply(ROC, timeperiod=t)

# Chaikin A/D Oscillator
for t in T:
    universe[f'{t:02}_ADOSC'] = (universe.groupby(level='symbol', group_keys=False)
     .apply(lambda x: ADOSC(x.high, x.low, x.close, x.volume, fastperiod=t-3, slowperiod=4+t)))

# Average Directional Movement Index
for t in T:
    universe[f'{t:02}_ADX'] = universe.groupby(level='symbol', group_keys=False).apply(lambda x:
                                                        ADX(x.high, x.low, x.close, timeperiod=t))
universe.drop(ohlcv, axis=1).to_hdf('data/universe_data.h5', 'features')

In [17]:
# Compute Historical Returns
by_sym = universe.groupby(level='symbol').close
for t in [1,5]:
    universe[f'r{t:02}'] = by_sym.pct_change(t)

# Remove outliers
universe[[f'r{t:02}' for t in [1, 5]]].describe()
outliers = universe[universe.r01>1].index.get_level_values('symbol').unique()
print(f'outliers length: {len(outliers)}')
universe = universe.drop(outliers, level='symbol')

outliers length: 14


In [18]:
# Historical return quantiles
for t in [1, 5]:
    #  add dropna() function for empty axis call
    universe[f'r{t:02}dec'] = (universe[f'r{t:02}'].dropna().groupby(level='date')
             .apply(lambda x: pd.qcut(x, q=10, labels=False, duplicates='drop')))

In [2]:
# universe.to_hdf('data/temp_universe.h5', 'universe')
universe = pd.read_hdf('data/temp_universe.h5', 'universe')

In [3]:
factor_data = (web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench',
                              start=START)[0].rename(columns={'Mkt-RF': 'Market'}))
factor_data.index.names = ['date']
factor_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5868 entries, 2000-01-03 to 2023-04-28
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Market  5868 non-null   float64
 1   SMB     5868 non-null   float64
 2   HML     5868 non-null   float64
 3   RMW     5868 non-null   float64
 4   CMA     5868 non-null   float64
 5   RF      5868 non-null   float64
dtypes: float64(6)
memory usage: 320.9 KB


In [4]:
windows = list(range(15, 90, 5))
len(windows)

15

In [5]:
t = 1
ret = f'r{t:02}'
factors = ['Market', 'SMB', 'HML', 'RMW', 'CMA']
windows = list(range(15, 90, 5))
for window in windows:
    # print(window)
    betas = []
    for symbol, data in universe.groupby(level='symbol'):
        print(f'Current Running Info - window: {window} | symbol: {symbol}')
        model_data = data[[ret]].merge(factor_data, on='date').dropna()
        model_data[ret] -= model_data.RF

        rolling_ols = RollingOLS(endog=model_data[ret],
                                 exog=sm.add_constant(model_data[factors]), window=window)
        factor_model = rolling_ols.fit(params_only=True).params.drop('const', axis=1)
        result = factor_model.assign(symbol=symbol).set_index('symbol', append=True)
        betas.append(result)
    betas = pd.concat(betas).rename(columns=lambda x: f'{window:02}_{x}')
    universe = universe.join(betas)

Current Running Info - window: 15 | symbol: A
Current Running Info - window: 15 | symbol: AAL
Current Running Info - window: 15 | symbol: AAP
Current Running Info - window: 15 | symbol: AAPL
Current Running Info - window: 15 | symbol: ABBV
Current Running Info - window: 15 | symbol: ABC
Current Running Info - window: 15 | symbol: ABT
Current Running Info - window: 15 | symbol: ACAS
Current Running Info - window: 15 | symbol: ACE
Current Running Info - window: 15 | symbol: ACI
Current Running Info - window: 15 | symbol: ACN
Current Running Info - window: 15 | symbol: ACXM
Current Running Info - window: 15 | symbol: ADBE
Current Running Info - window: 15 | symbol: ADI
Current Running Info - window: 15 | symbol: ADM
Current Running Info - window: 15 | symbol: ADP
Current Running Info - window: 15 | symbol: ADS
Current Running Info - window: 15 | symbol: ADSK
Current Running Info - window: 15 | symbol: ADTN
Current Running Info - window: 15 | symbol: AEE
Current Running Info - window: 15 |

In [9]:
# Compute Forward Returns
for t in [1, 5]:
    universe[f'r{t:02}_fwd'] = universe.groupby(level='symbol')[f'r{t:02}'].shift(-t)
    universe[f'r{t:02}dec_fwd'] = universe.groupby(level='symbol')[f'r{t:02}dec'].shift(-t)

# Store Model Data

universe = universe.drop(ohlcv, axis=1)
universe.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2536336 entries, ('BRK_A', Timestamp('2000-01-03 00:00:00')) to ('BWLD', Timestamp('2018-02-05 00:00:00'))
Columns: 308 entries, 06_RSI to r05dec_fwd
dtypes: float64(308)
memory usage: 5.8+ GB


In [10]:
drop_cols = ['r01', 'r01dec', 'r05',  'r05dec']
outcomes = universe.filter(like='_fwd').columns
universe = universe.sort_index()

In [11]:
with pd.HDFStore('data/universe_data.h5') as store:
    store.put('features', universe.drop(drop_cols, axis=1).drop(outcomes, axis=1).loc[idx[:, '2001':], :])
    store.put('targets', universe.loc[idx[:, '2001':], outcomes])

In [12]:
universe.sample(10)

,,06_RSI,07_RSI,08_RSI,09_RSI,10_RSI,11_RSI,12_RSI,13_RSI,14_RSI,15_RSI,...,80_CMA,85_Market,85_SMB,85_HML,85_RMW,85_CMA,r01_fwd,r01dec_fwd,r05_fwd,r05dec_fwd
symbol,date,,,,,,,,,,,,,,,,,,,,,
KMI,2015-06-12,8.713382,10.020615,11.315844,12.613166,13.911916,15.205370,16.485110,17.743158,18.972889,20.169278,...,0.001057,0.005256,-0.005957,-0.000501,-0.004660,-0.000134,0.011305,9.0,0.015930,6.0
VMC,2010-02-25,49.745682,49.085303,48.063121,47.002444,46.037182,45.212282,44.532747,43.987069,43.558189,43.228417,...,0.002939,0.015113,-0.000800,-0.005711,-0.005104,0.004422,-0.025589,0.0,-0.017508,0.0
KO,2007-05-08,66.214983,68.548652,70.253789,71.501568,72.405089,73.044305,73.478423,73.752594,73.901862,73.953703,...,0.006587,0.006461,-0.001981,-0.001036,0.001388,0.005610,0.000754,3.0,-0.011494,4.0
HCN,2017-05-08,38.404957,38.423582,39.017760,39.898490,40.902807,41.938446,42.953768,43.920848,44.825709,45.662563,...,0.009088,0.002183,-0.002680,-0.007463,0.003355,0.009710,-0.009897,1.0,0.027826,8.0
DRIV,2006-03-27,65.860153,66.530388,67.077024,67.519238,67.873463,68.152537,68.366362,68.522676,68.627705,68.686651,...,-0.007743,0.008052,0.004854,-0.033244,0.008136,-0.005608,-0.009007,3.0,0.028406,8.0
CAM,2007-04-12,76.786051,75.772994,74.952632,74.249941,73.624613,73.053486,72.522412,72.022369,71.547449,71.093714,...,-0.026013,0.007426,-0.000605,0.000189,0.024451,-0.026296,-0.005023,2.0,-0.014764,1.0
HES,2002-04-01,80.435940,79.814681,79.370568,79.027262,78.736248,78.467952,78.204907,77.937114,77.659105,77.368147,...,0.014506,0.009734,-0.006273,-0.009323,-0.004503,0.014923,0.021126,9.0,-0.017895,4.0
FOXA,2018-03-01,33.483223,35.960019,37.869547,39.399821,40.670273,41.757131,42.709590,43.559867,44.329453,45.033006,...,0.010358,0.008469,-0.001804,0.010424,-0.007196,0.007460,0.012528,7.0,0.035078,6.0
SANM,2008-01-23,47.890392,46.356451,45.263367,44.462249,43.860767,43.399739,43.040207,42.755850,42.528404,42.344845,...,0.034714,0.016042,0.008827,0.010074,-0.014793,0.034583,-0.035461,0.0,0.014184,4.0
